In [ ]:
class TreeNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.height = 1

class AvlTree:
    def __init__(self):
        self.root = None

    def height(self, node):
        if node is None:
            return 0
        return node.height

    def balance_factor(self, node):
        if node is None:
            return 0
        return self.height(node.left) - self.height(node.right)

    def rotate_left(self, x):
        y = x.right
        T2 = y.left
        y.left = x
        x.right = T2
        x.height = 1 + max(self.height(x.left), self.height(x.right))
        y.height = 1 + max(self.height(y.left), self.height(y.right))
        return y

    def rotate_right(self, y):
        x = y.left
        T2 = x.right
        x.right = y
        y.left = T2
        y.height = 1 + max(self.height(y.left), self.height(y.right))
        x.height = 1 + max(self.height(x.left), self.height(x.right))
        return x

    def rebalance(self, node):
        node.height = 1 + max(self.height(node.left), self.height(node.right))
        balance = self.balance_factor(node)
        if balance > 1:
            if self.balance_factor(node.left) < 0:
                node.left = self.rotate_left(node.left)
            return self.rotate_right(node)
        if balance < -1:
            if self.balance_factor(node.right) > 0:
                node.right = self.rotate_right(node.right)
            return self.rotate_left(node)
        return node

    def insert(self, key):
        def insert_helper(node, key):
            if node is None:
                return TreeNode(key)
            if key < node.key:
                node.left = insert_helper(node.left, key)
            else:
                node.right = insert_helper(node.right, key)
            return self.rebalance(node)
        self.root = insert_helper(self.root, key)

    def search(self, key):
        node = self.root
        while node is not None:
            if node.key == key:
                return True
            elif key < node.key:
                node = node.left
            else:
                node = node.right
        return False

    def delete(self, key):
        def delete_helper(node, key):
            if node is None:
                return node
            elif key < node.key:
                node.left = delete_helper(node.left, key)
            elif key > node.key:
                node.right = delete_helper(node.right, key)
            else:
                if node.left is None:
                    return node.right
                elif node.right is None:
                    return node.left
                else:
                    temp = self.min_value_node(node.right)
                    node.key = temp.key
                    node.right = delete_helper(node.right, temp.key)
            return self.rebalance(node)
        self.root = delete_helper(self.root, key)

    def min_value_node(self, node):
        current = node
        while current.left is not None:
            current = current.left
        return current

# Testws
tree = AvlTree()
tree.insert(10)
tree.insert(20)
tree.insert(30)
tree.insert(40)
tree.insert(50)
tree.insert(25)
tree.delete(30)
assert tree.search(30) == False
assert tree.search(25) == True

In [ ]:
class RabinKarp:
    def __init__(self, pattern):
        self.pattern = pattern
        self.pattern_len = len(pattern)
        self.pattern_hash = self._polynomial_hash(pattern)
        
    def search(self, text):
        n = len(text)
        if n < self.pattern_len:
            return -1
        
        # Compute initial hash for the first window
        text_hash = self._polynomial_hash(text[0:self.pattern_len])
        if text_hash == self.pattern_hash and text[0:self.pattern_len] == self.pattern:
            return 0

        # Compute hash for subsequent windows using rolling hash
        for i in range(1, n - self.pattern_len + 1):
            text_hash = self._rolling_hash(text[i-1], text[i+self.pattern_len-1], text_hash)
            if text_hash == self.pattern_hash and text[i:i+self.pattern_len] == self.pattern:
                return i
        
        return -1

    def _polynomial_hash(self, string):
        p = 31  # prime number for hashing
        m = 10**9 + 9  # large prime number
        hash_value = 0
        for c in string:
            hash_value = (hash_value * p + ord(c)) % m
        return hash_value

    def _rolling_hash(self, left_char, right_char, hash_value):
        p = 31  # prime number for hashing
        m = 10**9 + 9  # large prime number for modulo
        old_value = ord(left_char) * pow(p, self.pattern_len-1, m)
        new_value = ord(right_char)
        return ((hash_value - old_value) * p + new_value) % m

# Testing Rabin-Karp algorithm
rk = RabinKarp("AAA")
assert rk.search("AAATTTCGC") == 0
assert rk.search("CGCGCGC") == -1